In [1]:
import os
import logging
from pathlib import Path
import datetime

from tqdm import tqdm
import numpy as np
import math
import random

import torch
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt

import cv2

import config
import dataset

# Setup Warnings as Error to catch them

In [2]:
import warnings
warnings.filterwarnings("error")

# Load Train Dataset to catch the error

In [3]:
BATCH_SIZE = 1
NUM_WORKERS = 1

In [4]:
train_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(p=0.4),
            A.HueSaturationValue(hue_shift_limit=10, p=0.2),
            A.Blur(blur_limit=(3,3), p=0.3),
            A.CLAHE(clip_limit=2.0, p=0.3),
            A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        ], p=0.9),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.Resize(config.IMG_H, config.IMG_W, p=1),
        #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1),
        ToTensorV2(p=1),
    ]
)

try:
    # TRAIN DATASET
    print("\nTRAIN DFIRE dataset")
    train_dataset = dataset.DFireDataset(
        img_h = config.IMG_H,
        img_w = config.IMG_W,
        img_dir = config.TRAIN_IMG_DIR,
        label_dir = config.TRAIN_LABEL_DIR,
        num_classes=config.N_CLASSES,
        ds_len = config.DS_LEN,
        transform=train_transform)
    
    print(f'Train dataset len: {len(train_dataset)}')
    
    
    # LOADERS
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              num_workers=1,
                              pin_memory=config.PIN_MEMORY,
                              shuffle=True,
                              drop_last=True)
except RuntimeWarning:
    raise SystemExit("Wrong Image Found")


TRAIN DFIRE dataset


Corrupt JPEG data: 1 extraneous bytes before marker 0xd9


DFire Removed wrong images: 0
DFire empty images: 7833
DFire only smoke images: 4680
DFire only fire images: 943
DFire smoke and fire images: 3763
Train dataset len: 17219


# Iterate trying to find wrong image

In [5]:
# i = 0
# try:
#     for img, label in train_loader:
#         i += 1    
# except RuntimeWarning:
#     raise SystemExit("Wrong Image Found")   